In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
dataset = tfds.load("imdb_reviews",as_supervised=True)

In [ ]:
train_dataset, test_dataset = dataset['train'],dataset['test']

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
VOCAB_SIZE = 1000

In [ ]:
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)

In [ ]:
encoder.adapt(train_dataset.map(lambda text,label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [ ]:
model = tf.keras.Sequential([encoder,tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),output_dim=64,mask_zero=True),tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),tf.keras.layers.Dense(64,activation='relu'),tf.keras.layers.Dense(1)])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(1e-4),metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset,validation_steps=30)